# inceptionV3 모델

In [1]:
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import DenseNet121, MobileNetV2, DenseNet201, EfficientNetB0, EfficientNetB1, InceptionV3, MobileNetV3Large, ResNet152V2, ResNet50, ResNet50V2, VGG19, VGG16, Xception

# 이미지 크기 설정
img_size = (224, 224)

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    selected_images = random.sample(image_paths, 1000)

    for img_path in selected_images:
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)

        if brightness:
            img_array = cv2.convertScaleAbs(img_array, alpha=1.2, beta=30)

        img_array = img_array.astype('float32') / 255.0

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)

# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 이미지 데이터 로드 및 전처리
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("C:\\Users\\김현\\Final_Project\\crawled_img", category)
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]
    
    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 사전 학습된 모델들
models = {
    "DenseNet121": DenseNet121,
    "MobileNetV2": MobileNetV2,
    "DenseNet201": DenseNet201,
    "EfficientNetB0": EfficientNetB0,
    "EfficientNetB1": EfficientNetB1,
    "InceptionV3": InceptionV3,
    "MobileNetV3Large": MobileNetV3Large,
    "ResNet152V2": ResNet152V2,
    "ResNet50": ResNet50,
    "ResNet50V2": ResNet50V2,
    "VGG19": VGG19,
    "VGG16": VGG16,
    "Xception": Xception
}

# 모델 선택
base_model = models["InceptionV3"](weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 새로운 모델 생성
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# EarlyStopping 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


87910968/87910968 [==============================] - 12s 0us/step
Epoch 1/50
150/150 [==============================] - 1743s 11s/step - loss: 0.8869 - accuracy: 0.7165 - val_loss: 3.6187 - val_accuracy: 0.4225
Epoch 2/50
150/150 [==============================] - 1721s 11s/step - loss: 0.6071 - accuracy: 0.8110 - val_loss: 1.4630 - val_accuracy: 0.6033
Epoch 3/50
150/150 [==============================] - 1702s 11s/step - loss: 0.4910 - accuracy: 0.8535 - val_loss: 1.4250 - val_accuracy: 0.5833
Epoch 4/50
150/150 [==============================] - 1694s 11s/step - loss: 0.4439 - accuracy: 0.8637 - val_loss: 1.1359 - val_accuracy: 0.6400
Epoch 5/50
150/150 [==============================] - 1691s 11s/step - loss: 0.4016 - accuracy: 0.8748 - val_loss: 1.0966 - val_accuracy: 0.6633
Epoch 6/50
150/150 [==============================] - 1676s 11s/step - loss: 0.3584 - accuracy: 0.8904 - val_loss: 2.1149 - val_accuracy: 0.4950
Epoch 7/50
150/150 [==============================] - 1685s 11s/

In [2]:
# 테스트 세트에서 모델 평가
evaluation_results = model.evaluate(X_test, y_test)

# 평가 결과 출력
print(f'테스트 손실: {evaluation_results[0]:.4f}')
print(f'테스트 정확도: {evaluation_results[1]*100:.2f}%')


38/38 [==============================] - 83s 2s/step - loss: 0.7962 - accuracy: 0.8125
테스트 손실: 0.7962
테스트 정확도: 81.25%


In [3]:
from tensorflow.keras.models import save_model, load_model

# 전체 모델 저장
model.save("C:/Users/김현/Final_Project/TL_inception.h5")

C:\Users\김현\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
